<a href="https://colab.research.google.com/github/Ruchiraongithub/Projects/blob/main/Music_Recomandation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd

In [15]:
df=pd.read_csv("/content/spotify_millsongdata.csv.zip")

In [16]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [17]:
df.shape

(57650, 4)

In [18]:
df.isnull().sum()

,0
artist,0
song,0
link,0
text,0


In [19]:
df =df.sample(5000).drop("link",axis=1).reset_index(drop=True)

In [20]:
df.head(5)

,artist,song,text
0,Gary Numan,I Dream Of Wires,I am the final silence \r\nThe last electrici...
1,Loretta Lynn,Race Is On,Well I feel tears wellin' up cold and deep ins...
2,Thin Lizzy,Diddy Levine,In the later forties \r\nWhen Diddy Levine li...
3,Doobie Brothers,Little Darling,"Eddie holland, lamont dozier, and brian hollan..."
4,Don Henley,Sit Down You're Rockin' The Boat,I dreamed last night \r\nI was on the boat to...


In [21]:
df['text']= df['text'].str.lower().replace(r'^\w\s',' ').replace(r'\n',' ',regex=True)

In [22]:
df

,artist,song,text
0,Gary Numan,I Dream Of Wires,i am the final silence \r the last electricia...
1,Loretta Lynn,Race Is On,well i feel tears wellin' up cold and deep ins...
2,Thin Lizzy,Diddy Levine,in the later forties \r when diddy levine liv...
3,Doobie Brothers,Little Darling,"eddie holland, lamont dozier, and brian hollan..."
4,Don Henley,Sit Down You're Rockin' The Boat,i dreamed last night \r i was on the boat to ...
...,...,...,...
4995,Townes Van Zandt,Brother Flower,"brother flower, are you listenin'? \r let me ..."
4996,Joni Mitchell,The Dry Cleaner From Des Moines,i'm down to a roll of dimes \r i'm stalking t...
4997,Nine Inch Nails,Came Back Haunted,ha-ha-haunted \r \r the throat is deep and ...
4998,Planetshakers,Never Stop,all my hopes \r and all my dreams \r god i l...


In [23]:
import nltk
from nltk.stem.porter import PorterStemmer

In [24]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [25]:
stemmer = PorterStemmer()

In [26]:
def token(txt):
  token = nltk.word_tokenize(txt)
  a =[stemmer.stem(w) for w in token]
  return " ".join(a)

In [27]:
df['text'].apply(lambda x: token(x))

,text
0,i am the final silenc the last electrician ali...
1,well i feel tear wellin ' up cold and deep ins...
2,in the later forti when diddi levin live with ...
3,"eddi holland , lamont dozier , and brian holla..."
4,i dream last night i wa on the boat to heaven ...
...,...
4995,"brother flower , are you listenin ' ? let me s..."
4996,i 'm down to a roll of dime i 'm stalk the slo...
4997,ha-ha-haunt the throat is deep and the mouth i...
4998,all my hope and all my dream god i lay them at...


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [29]:
tfid = TfidfVectorizer(analyzer='word',stop_words='english')

In [30]:
matrix = tfid.fit_transform(df['text'])

In [31]:
similar= cosine_similarity(matrix)

In [35]:
def recommender(song_name):
  idx = df[df['song']==song_name].index[0]
  distance = sorted(list(enumerate(similar[idx])),reverse=True,key=lambda x:x[1])
  song = []
  for s_id in distance[1:21]:
    song.append(df.iloc[s_id[0]].song)
  return song

In [37]:
def recommender(song_name):
  # Check if song exists in DataFrame
  if df[df['song']==song_name].empty:
    print(f"Song '{song_name}' not found in the dataset.")
    return []  # Return empty list if song not found

  idx = df[df['song']==song_name].index[0]
  distance = sorted(list(enumerate(similar[idx])),reverse=True,key=lambda x:x[1])
  song = []
  for s_id in distance[1:21]:
    song.append(df.iloc[s_id[0]].song)
  return song

In [45]:
input_song = input("Enter a song name: ")

# Get recommended songs
recommended_songs = recommender(input_song)

# Print the recommended songs
if recommended_songs:
    print(f"Songs similar to '{input_song}':")
    for i, song in enumerate(recommended_songs, 1):
        print(f"{i}. {song}")
else:
    print("No similar songs found.")


Enter a song name: Day By Day
Songs similar to 'Day By Day':
1. Day After Day
2. Airplane
3. Dance Into The Light
4. Shoot You Down
5. Closer You And I
6. Don't Wait Until Tomorrow
7. Long As I Can See The Light
8. Love Won't Wait
9. Eventually
10. Dusty
11. Night And Day
12. Rise And Shine
13. All I Want
14. Day By Day
15. Everyday
16. Day In-Day Out
17. Fall At Your Feet
18. One Step Closer
19. To The Morning
20. Goin' Home
